### 1. 安装HuggingFace 并下载模型到本地

In [ ]:
!pip install huggingface-hub -Uqq
!pip install -U sagemaker
!pip install -U boto
!pip install -U botocore

In [ ]:
#%pip install sagemaker pip --upgrade  --quiet
# Note the following may error depending on which awscli is installed in your jupyter kernel, 
# but that is ok 
#%pip install ./code/botocore-1.29.157-py3-none-any.whl ./code/boto3-1.26.157-py3-none-any.whl --force

In [1]:
!rm -rf ./LLM_chatglm3_stream_model

In [26]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./LLM_chatglm3_stream_model")
local_model_path.mkdir(exist_ok=True)
model_name = "THUDM/chatglm3-6b"
commit_hash = "fc3235f807ef5527af598c05f04f2ffd17f48bab"

In [27]:
snapshot_download(repo_id=model_name, cache_dir=local_model_path)

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.83G [00:00<?, ?B/s]

(…)8c05f04f2ffd17f48bab/modeling_chatglm.py:   0%|          | 0.00/55.6k [00:00<?, ?B/s]

(…)7af598c05f04f2ffd17f48bab/.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

(…)5527af598c05f04f2ffd17f48bab/config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

...

(…)af598c05f04f2ffd17f48bab/quantization.py:   0%|          | 0.00/14.7k [00:00<?, ?B/s]

(…)f04f2ffd17f48bab/tokenization_chatglm.py:   0%|          | 0.00/11.3k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

(…)05f04f2ffd17f48bab/tokenizer_config.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

'LLM_chatglm3_stream_model/models--THUDM--chatglm3-6b/snapshots/fc3235f807ef5527af598c05f04f2ffd17f48bab'

### 2. 把模型拷贝到S3为后续部署做准备

In [28]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [29]:
s3_model_prefix = "LLM-RAG/workshop/LLM_chatglm3_stream_model"  # folder where model checkpoint will go
model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/LLM_chatglm3_stream_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/LLM_chatglm3_stream_deploy_code
model_snapshot_path: LLM_chatglm3_stream_model/models--THUDM--chatglm3-6b/snapshots/fc3235f807ef5527af598c05f04f2ffd17f48bab


In [30]:
!aws s3 rm --recursive s3://{bucket}/{s3_model_prefix}
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}


delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/.gitattributes
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/MODEL_LICENSE
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/README.md
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/configuration_chatglm.py
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/config.json
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/modeling_chatglm.py
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/pytorch_model.bin.index.json
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/quantization.py
delete: s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/tokenization_chatglm.py
delete: s3://sagemaker-us-west-2-6

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [31]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
)

#中国区需要替换为下面的image_uri
# inference_image_uri = (
#     f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.21.0-deepspeed0.8.3-cu117"
# )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [32]:
!mkdir -p LLM_chatglm3_stream_deploy_code

In [33]:
%%writefile LLM_chatglm3_stream_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os

from transformers import pipeline, AutoModel, AutoTokenizer
model = None
tokenizer = None
STOP_flag = "[DONE]"

DEVICE = "cuda"
DEVICE_ID = "0"
CUDA_DEVICE = f"{DEVICE}:{DEVICE_ID}" if DEVICE_ID else DEVICE
def torch_gc():
    if torch.cuda.is_available():
        with torch.cuda.device(CUDA_DEVICE):
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            
def load_model(properties):
    global tokenizer,model
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")
    
    tokenizer = AutoTokenizer.from_pretrained(model_location, trust_remote_code=True)
   
    model = AutoModel.from_pretrained(model_location, trust_remote_code=True).half().cuda()
    
    model.requires_grad_(False)
    model.eval()
    
    return model, tokenizer



def stream_items(prompt, history, max_length, top_p, temperature):
    global model, tokenizer
    size = 0
    response = ""
    for response, history in model.stream_chat(tokenizer, prompt, history=history, max_length=max_length, top_p=top_p,
                                               temperature=temperature):
        this_response = response[size:]
        history = [list(h) for h in history]
        size = len(response)
        stream_buffer = { "outputs":this_response}
        yield stream_buffer
    ## stop
    # yield {"query": prompt, "outputs": STOP_flag, "response": response, "history": history, "finished": True}
    


def handle(inputs: Input):
    global model, tokenizer
    if not model:
        model, tokenizer = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = data["inputs"]
    params = data["parameters"]
    history = data["history"]
    print(f'input prompt:{input_sentences}')    
    outputs = Output()
    outputs.add_property("content-type", "application/jsonlines")
    outputs.add_stream_content(stream_items(input_sentences,history=history,**params))
    return outputs


Overwriting LLM_chatglm3_stream_deploy_code/model.py


In [34]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

option.s3url ==> s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/


#### Note: option.s3url 需要按照自己的账号进行修改, 可以拷贝上一个cell的输出

In [35]:
%%writefile LLM_chatglm3_stream_deploy_code/serving.properties
engine=Python
option.tensor_parallel_degree=1
option.enable_streaming=True
option.predict_timeout=240
option.s3url=s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/


Overwriting LLM_chatglm3_stream_deploy_code/serving.properties


In [36]:
!aws s3 ls s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_model/ --human

2023-10-30 00:51:55    1.5 KiB .gitattributes
2023-10-30 00:51:55    4.0 KiB MODEL_LICENSE
2023-10-30 00:51:55    5.4 KiB README.md
2023-10-30 00:51:55    1.3 KiB config.json
2023-10-30 00:51:55    2.3 KiB configuration_chatglm.py
2023-10-30 00:51:55   54.3 KiB modeling_chatglm.py
2023-10-30 00:51:55    1.7 GiB pytorch_model-00001-of-00007.bin
2023-10-30 00:51:55    1.8 GiB pytorch_model-00002-of-00007.bin
2023-10-30 00:51:55    1.8 GiB pytorch_model-00003-of-00007.bin
2023-10-30 00:51:55    1.7 GiB pytorch_model-00004-of-00007.bin
2023-10-30 00:51:55    1.8 GiB pytorch_model-00005-of-00007.bin
2023-10-30 00:53:08    1.8 GiB pytorch_model-00006-of-00007.bin
2023-10-30 00:53:08 1004.0 MiB pytorch_model-00007-of-00007.bin
2023-10-30 00:53:12   20.0 KiB pytorch_model.bin.index.json
2023-10-30 00:53:13   14.3 KiB quantization.py
2023-10-30 00:53:13   11.0 KiB tokenization_chatglm.py
2023-10-30 00:53:13  994.5 KiB tokenizer.model
2023-10-30 00:53:13  244 Bytes tokenizer_config.json


#### 注意: 必须把transformers升级到4.27.1以上，否则会出现 [Issue344](https://github.com/THUDM/ChatGLM-6B/issues/344)

In [37]:
%%writefile LLM_chatglm3_stream_deploy_code/requirements.txt
transformers==4.30.2

Overwriting LLM_chatglm3_stream_deploy_code/requirements.txt


In [38]:
!rm model.tar.gz
!cd LLM_chatglm3_stream_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf model.tar.gz LLM_chatglm3_stream_deploy_code

LLM_chatglm3_stream_deploy_code/
LLM_chatglm3_stream_deploy_code/model.py
LLM_chatglm3_stream_deploy_code/serving.properties
LLM_chatglm3_stream_deploy_code/requirements.txt


In [39]:
s3_code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_chatglm3_stream_deploy_code/model.tar.gz


### 4. 创建模型 & 创建endpoint

In [47]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base(f"chatglm-stream") # Append a timestamp to the provided string
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact,
        'Environment': {
            'SERVING_OPTS': '-Dai.djl.logging.level=DEBUG'
        },
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

chatglm-stream-2023-10-30-02-34-02-040
Image going to be used is ---- > 763104351884.dkr.ecr.us-west-2.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-west-2:687912291502:model/chatglm-stream-2023-10-30-02-34-02-040


In [48]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

#Note: ml.g4dn.2xlarge 也可以选择
endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 10*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:687912291502:endpoint-config/chatglm-stream-2023-10-30-02-34-02-040-config',
 'ResponseMetadata': {'RequestId': '5c42c51f-cfa7-4e52-a9eb-17221ca70f8f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5c42c51f-cfa7-4e52-a9eb-17221ca70f8f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '126',
   'date': 'Mon, 30 Oct 2023 02:34:02 GMT'},
  'RetryAttempts': 0}}

In [49]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-west-2:687912291502:endpoint/chatglm-stream-2023-10-30-02-34-02-040-endpoint


#### 持续检测模型部署进度

In [50]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-west-2:687912291502:endpoint/chatglm-stream-2023-10-30-02-34-02-040-endpoint
Status: InService


### 5. 模型测试

In [51]:
%%time
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 2048,
  "temperature": 0.01,
  "top_p":0.8
}

CPU times: user 2.92 ms, sys: 0 ns, total: 2.92 ms
Wall time: 2.4 ms


In [54]:
import io

endpoint_name="chatglm-stream-2023-10-30-02-34-02-040-endpoint"
class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

In [57]:
prompts1 = """帮我写一个童话故事，400字左右，主题是丢失的宠物冒险： 描述一个小朋友的宠物走失了，主人为了找回宠物踏上了冒险之旅，他们一起经历了什么？"""
# prompts1 = """write a 500 words story about scifiction"""
response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompts1,
                "parameters": parameters,
                "history" : []
            }
            ),
            ContentType="application/json",
        )

event_stream = response_model['Body']
scanner = StreamScanner()
for event in event_stream:
    #print(event)
    scanner.write(event['PayloadPart']['Bytes'])
    for line in scanner.readlines():
        try:
            resp = json.loads(line)
            print(resp)
            # print(resp.get("outputs")['outputs'], end='')
        except Exception as e:
            # print(line)
            continue

{'outputs': {'outputs': ''}}
{'outputs': {'outputs': ''}}
{'outputs': {'outputs': '很久'}}
{'outputs': {'outputs': '很久'}}
{'outputs': {'outputs': '以前'}}
{'outputs': {'outputs': ','}}
{'outputs': {'outputs': '有个'}}
{'outputs': {'outputs': '小朋友'}}
{'outputs': {'outputs': '叫'}}
{'outputs': {'outputs': '小明'}}
{'outputs': {'outputs': ','}}
{'outputs': {'outputs': '他'}}
{'outputs': {'outputs': '有一'}}
{'outputs': {'outputs': '只'}}
{'outputs': {'outputs': '可爱的'}}
{'outputs': {'outputs': '宠物'}}
{'outputs': {'outputs': '——'}}
{'outputs': {'outputs': '一只'}}
{'outputs': {'outputs': '叫做'}}
{'outputs': {'outputs': '“'}}
{'outputs': {'outputs': '小白'}}
{'outputs': {'outputs': '”'}}
{'outputs': {'outputs': '的小'}}
{'outputs': {'outputs': '狗'}}
{'outputs': {'outputs': '。'}}
{'outputs': {'outputs': '小明'}}
{'outputs': {'outputs': '和小'}}
{'outputs': {'outputs': '白'}}
{'outputs': {'outputs': '一起'}}
{'outputs': {'outputs': '生活'}}
{'outputs': {'outputs': '得'}}
{'outputs': {'outputs': '非常'}}
{'outputs': {'outputs

#### 清除模型Endpoint和config

In [ ]:
!aws sagemaker delete-endpoint --endpoint-name  {endpoint_name}

In [ ]:
!aws sagemaker delete-endpoint-config --endpoint-config-name {endpoint_config_name}

In [ ]:
!aws sagemaker delete-model --model-name  {model_name}